# Initialising Your Agent as an Issuing Authority

This template walks you through the basic steps you need to take to configure your agent as an issuing authority on the Sovrin StagingNet. If using a different network you will need to update this template. The steps include:

* Writing your DID to the Sovrin StagingNet
* Accepting the Transaction Author Agreement
* Authoring schema to the ledger
* Authoring credential definitions for the schema this agent intends to issue
* Persisting Identifiers for use throughout the playground.

It is recommended that this initialisation notebook be run **once**. If you are following the default docker-compose services then your agents wallet storage will be persisted in a postgres database as long as you run `./manage stop` rather than `./manage down`. 



### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://general-medical-council-agent:3021 and an api key of adminApiKey


## Write DID to the Public Ledger

Note: if defined a ACAPY_WALLET_SEED value for your agent then this function will return a DID, but this DID still needs to be written to the ledger. If you did not define a seed you will need to create a DID first.

In [3]:
public_did_response = await agent_controller.wallet.get_public_did()

In [4]:
print(public_did_response)

{'result': {'did': 'TDAbSf3Uqebg8N4XvybMbg', 'verkey': 'FHYRgHt1Z7pZBEY5yMaTHoeLMTvbQ9R6qdxhJu69xEw2', 'posture': 'public'}}


In [5]:
if public_did_response["result"]:
    did_obj = public_did_response["result"]
else:
    create_did_response = await agent_controller.wallet.create_did()
    did_obj = create_did_response['result']
print("DID", did_obj)

DID {'did': 'TDAbSf3Uqebg8N4XvybMbg', 'verkey': 'FHYRgHt1Z7pZBEY5yMaTHoeLMTvbQ9R6qdxhJu69xEw2', 'posture': 'public'}


## Not Needed for Local VON Network

In [6]:
# # write new DID to Sovrin Stagingnet
# import requests
# import json 

# url = 'https://selfserve.sovrin.org/nym'

# payload = {"network":"stagingnet","did": did_obj["did"],"verkey":did_obj["verkey"],"paymentaddr":""}

# # Adding empty header as parameters are being sent in payload
# headers = {}

# r = requests.post(url, data=json.dumps(payload), headers=headers)
# print(r.json())

## Not Needed for Local VON Network

## Accept Transaction Author Agreement

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [7]:
# taa_response = await agent_controller.ledger.get_taa()
# TAA = taa_response['result']['taa_record']
# TAA['mechanism'] = "service_agreement"
# await agent_controller.ledger.accept_taa(TAA)

## Assign Agent Public DID if Not Set

Will only be ran if ACAPY_WALLET_SEED not initially set.

In [6]:
if did_obj["posture"] != "public":
    response = await agent_controller.wallet.assign_public_did(did_obj["did"])
print("Successfully intialised agent with Public DID : ", did_obj["did"])

Successfully intialised agent with Public DID :  TDAbSf3Uqebg8N4XvybMbg


## Writing GMC Licence Schema

Note that the headshot attribute will contain a base64 string representing an image of the doctor.

In [7]:
# Define you schema name - must be unique on the ledger
schema_name = "GMC Licence"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["GMC Number", "Responsible Officer GMC Number", "Licenced From", "Re-Validation Due", "Name", "Headshot", "DOB"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
gmc_schema_id = response["schema_id"]

## Writing The GMC Licence Definition

The GMC Licence is the foundational credential in a healthcare professionals career. All practicing doctors in the UK must have a licence and it is subject to review every 5 years.

For this reason we have chosen to make the GMC Licence a revocable credential, such that the GMC can publish anonymous on chain updates revoking a doctors licence to practice at any time.

Clearly there would need to be some regulations and governance around this, but likely these already exist as part of the General Medical Council's processes.

Using anonymous revocation is important because it removes the requirement for a doctor to reveal a unique identifier every time they present their licence. Note if a unique identifier is required, any likely it would be a lot of the time, the interaction can request disclosure of the GMC number that identifies the doctor.

In [8]:
# Tag and group specific credential definitions
tag = "default"

# Make Cred Def support revocation. Credentials issued using this definition will be able to be revoked.
support_revocation = True

cred_def_response = await agent_controller.definitions.write_cred_def(gmc_schema_id, tag, support_revocation)
gmc_cred_def_id = cred_def_response["credential_definition_id"]

## Writing Schema For Other Required Information Used in Doctor Onboarding

The GMC in this scenario has been identified as a key root of trust for the healthcare system. As such they are going to issue additional credentials that the healthcare professional can use when onboarding at hospitals. These are:

* Right to Work in the UK - The GMC must perform these checks before they can issue a licence
* DBS Check - Ensures the professional has no criminal record. 

**Note: This check seems to be on a per role basis, so possible the GMC is not the best authority. Another option could be the Scottish Deanery who employs and assigns placements to Junior Doctors throughout the UK**

Whether the GMC is the suitable entity to take on this role is an open question. However, they are already trusted by all hospitals and have regular contact with healthcare professionals throughout their career.

Furthermore, they complete these checks anyway so would seem to be in an ideal position to provide cryptographic evidence attesting to their successful completion.

These schema might be defined by another entity. Ideally Gov.UK. This is likely to take a long time.

In [9]:
# Define you schema name - must be unique on the ledger
schema_name = "UK Right to Work"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["Eligible", "Date Completed", "Expiration Date", "Address"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
rtw_schema_id = response["schema_id"]

In [10]:
# Define you schema name - must be unique on the ledger
schema_name = "DBS Check"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema

# The idea with the Certificate Reference Number is it could be looked up on the gov database to verify this information
# Rather than reproducing all the certification information which the GMC is not authorised to do
# This credential is evidence that a DBS check has taken place
attributes = ["Certificate Reference Number","Date Completed", "Expiration Date", "Type"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
dbs_schema_id = response["schema_id"]

## Write Credential Definitions

Note: These are not revocable credentials but they could be if needed

In [11]:
# Tag and group specific credential definitions
tag = "doctor-onboarding"

# Make Cred Def support revocation. Credentials issued using this definition will be able to be revoked.
support_revocation = False

cred_def_response = await agent_controller.definitions.write_cred_def(rtw_schema_id, tag, support_revocation)
rtw_cred_def_id = cred_def_response["credential_definition_id"]

In [12]:
# Tag and group specific credential definitions
tag = "doctor-onboarding"

# Make Cred Def support revocation. Credentials issued using this definition will be able to be revoked.
support_revocation = False

cred_def_response = await agent_controller.definitions.write_cred_def(dbs_schema_id, tag, support_revocation)
dbs_cred_def_id = cred_def_response["credential_definition_id"]

## Persist Identifiers for use throughout other business logic notebooks associated with this agent

The schema_id and cred_def_id value pairs are required whenever issuing credentials, and also can be used to constrain acceptable proof requests. In a real application these values might be stored in environment variables or most likely in a database. For notebooks we have found it easier to store as string values in a cell and then load these values into the jupyter store so that they can be fetched across multiple notebooks.

As such you are recommended to print out each of the schema and cred def identifiers used by your agent and copy them across to your **main** business logic notebook where you should store them in a variable and save them to the jupyter store. Remember, you should only be running this notebook once so having this logic in here will not be useful.


In [13]:
print(f"gmc_schema_id = '{gmc_schema_id}'")
print(f"gmc_cred_def_id = '{gmc_cred_def_id}'")

gmc_schema_id = 'TDAbSf3Uqebg8N4XvybMbg:2:GMC Licence:0.0.1'
gmc_cred_def_id = 'TDAbSf3Uqebg8N4XvybMbg:3:CL:12:default'


In [14]:
print(f"rtw_schema_id = '{rtw_schema_id}'")
print(f"rtw_cred_def_id = '{rtw_cred_def_id}'")

rtw_schema_id = 'TDAbSf3Uqebg8N4XvybMbg:2:UK Right to Work:0.0.1'
rtw_cred_def_id = 'TDAbSf3Uqebg8N4XvybMbg:3:CL:18:doctor-onboarding'


In [15]:
print(f"dbs_schema_id = '{dbs_schema_id}'")
print(f"dbs_cred_def_id = '{dbs_cred_def_id}'")

dbs_schema_id = 'TDAbSf3Uqebg8N4XvybMbg:2:DBS Check:0.0.1'
dbs_cred_def_id = 'TDAbSf3Uqebg8N4XvybMbg:3:CL:19:doctor-onboarding'


## Terminate Controller


In [16]:
await agent_controller.terminate()